### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [146]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

In [147]:
# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

# Check structure and contents of DataFrames
school_df.head()
student_df.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [148]:
# calculate total number of schools
school_count = len(school_df['school_name'].unique())

# calculate total number of students
student_count = len(student_df['Student ID'].unique())

# calculate total budget
budget = sum(school_df['budget'])

# calculate the average math score
avg_math = student_df['math_score'].mean()

# calculate the average reading score
avg_reading = student_df['reading_score'].mean()

# calculate the percentage of students with a passing math score (70 or greater)
math_df = student_df.loc[student_df['math_score'] >= 70]
passed_math = len(math_df) / student_count * 100

# calculate the percentage of students with a passing reading score (70 or greater)
reading_df = student_df.loc[student_df['reading_score'] >= 70]
passed_reading = len(reading_df) / student_count * 100

# calculate the percentage of students who passed math and reading (% overall passing)
overall_df = student_df.loc[(student_df['math_score'] >= 70) & (combined_df['reading_score'] >= 70)]
passed_overall = len(overall_df) / student_count * 100

# create a dataframe to hold the results
district_summary_df = pd.DataFrame({"Total Schools": [school_count],
                                    "Total Students": student_count,
                                    "Total Budget": budget,
                                    "Average Math Score": avg_math,
                                    "Average Reading Score": avg_reading,
                                    "% Passing Math": passed_math,
                                    "% Passing Reading": passed_reading,
                                    "% Overall Passing": passed_overall})

# optional: clean formatting
district_summary_df['Total Students'] = district_summary_df['Total Students'].map('{:,.0f}'.format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('${:,.2f}'.format)

district_summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [164]:
from functools import reduce

# summarize school info and budget
school_df['Per Student Budget'] = school_df['budget'] / school_df['size']
# school_df

# summarize math and reading averages by each school
school_scores_df = student_df[['school_name','math_score','reading_score']]
school_averages_df = school_scores_df.groupby('school_name').mean()
# school_averages_df

# count number of students per school who passed math and save to dataframe
math_school_s = math_df['school_name'].value_counts()
math_school_df = pd.Series.to_frame(math_school_s).reset_index()
math_school_df = math_school_df.rename(columns = {'index': 'school_name','school_name': 'math_passed'})
# math_school_df

# count number of students per school who passed reading and save to dataframe
reading_school_s = reading_df['school_name'].value_counts()
reading_school_df = pd.Series.to_frame(reading_school_s).reset_index()
reading_school_df = reading_school_df.rename(columns = {'index': 'school_name','school_name': 'reading_passed'})
# reading_school_df

# count number of students per school who passed math and reading and save to dataframe
overall_school_s = overall_df['school_name'].value_counts()
overall_school_df = pd.Series.to_frame(overall_school_s).reset_index()
overall_school_df = overall_school_df.rename(columns = {'index': 'school_name','school_name': 'overall_passed'})
# overall_school_df

# use reduce to merge multiple dataframes in one step
dfs_to_merge = [school_df, school_averages_df, math_school_df, reading_school_df, overall_school_df]
merged_df = reduce(lambda left,right: pd.merge(left,right, on=['school_name'], how='outer'),dfs_to_merge)

# calculate percentage of students passing math and/or reading
merged_df['% Passing Math'] = merged_df['math_passed'] / merged_df['size'] * 100
merged_df['% Passing Reading'] = merged_df['reading_passed'] / merged_df['size'] * 100
merged_df['% Overall Passing'] = merged_df['overall_passed'] / merged_df['size'] * 100
# merged_df

# select final columns, rename headers, and format values
subset_df = merged_df[['school_name','type','size','budget','Per Student Budget','math_score','reading_score','% Passing Math','% Passing Reading','% Overall Passing']]
sorted_df = subset_df.sort_values('school_name', ascending=True).reset_index(drop=True)
sorted_df['budget'] = sorted_df['budget'].map('${:,.2f}'.format)
sorted_df['Per Student Budget'] = sorted_df['Per Student Budget'].map('${:,.2f}'.format)
school_summary_df = sorted_df.rename(columns={'school_name': 'School Name',
                                                      'type': 'School Type',
                                                      'size': 'Total Students',
                                                      'budget': 'Total School Budget',
                                                      'math_score': 'Average Math Score',
                                                      'reading_score': 'Average Reading Score'})
school_summary_df



,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type